# Generalized Shallow Water Equations

In [ ]:
load('GeneralizedShallowWaterVariables.sage')

## Zeroth Order/Constant System/Shallow Water Equations

In [26]:
set_generalized_shallow_water_variables(0)
eig = A_p.eigenvectors_right()
eigenvalues = [e[0].simplify_full() for e in eig]
eigenvectors = [[ev.simplify_full() for ev in e[1]] for e in eig]
print(eigenvalues)
print(eigenvectors)

[u - sqrt(g*h), u + sqrt(g*h)]
[[(1, u - sqrt(g*h))], [(1, u + sqrt(g*h))]]


In [27]:
A_p.eigenvectors_left()

[(u - sqrt(g*h), [(1, (u - sqrt(g*h))/(g*h - u^2))], 1),
 (u + sqrt(g*h), [(1, (u + sqrt(g*h))/(g*h - u^2))], 1)]

## First Order/Linear System

In [14]:
set_generalized_shallow_water_variables(1)
eig = A_p.eigenvectors_right()
eigenvalues = [e[0].simplify_full() for e in eig]
eigenvectors = [[ev.simplify_full() for ev in e[1]] for e in eig]
print(eigenvalues)
print(eigenvectors)

[u - sqrt(g*h + s^2), u + sqrt(g*h + s^2), u]
[[(1, u - sqrt(g*h + s^2), 2*s)], [(1, u + sqrt(g*h + s^2), 2*s)], [(1, u, -1/2*(3*g*h - s^2)/s)]]


## Second Order/Quadratic

In [17]:
set_generalized_shallow_water_variables(2)
eig = A_p.eigenvectors_right()
eigenvalues = [e[0].simplify_full() for e in eig]
eigenvectors = [[ev.simplify_full() for ev in e[1]] for e in eig]
print(eigenvalues)
print(eigenvectors)

[5/14*k + u - 1/210*sqrt(15)*sqrt((8575*g^2*h^2 + 11340*g*h*k^2 + 20412*k^4 + 32928*s^4 + 84*(490*g*h - 243*k^2)*s^2 + (-42875*g^3*h^3 + 103950*g^2*h^2*k^2 + 131220*g*h*k^4 + 157464*k^6 + 296352*s^6 + 504*(980*g*h + 27*k^2)*s^4 + 18*(8575*g^2*h^2 - 85050*g*h*k^2 - 34992*k^4)*s^2 + 90*sqrt(-6002500/3*g^5*h^5*k^2 - 2866500*g^4*h^4*k^4 - 3110400*g^3*h^3*k^6 - 524880*g^2*h^2*k^8 - 30118144/15*s^12 - 307328/5*(196*g*h - 405*k^2)*s^10 - 784/15*(552230*g^2*h^2 + 1222452*g*h*k^2 + 1622025*k^4)*s^8 - 8/15*(64706950*g^3*h^3 + 303714495*g^2*h^2*k^2 + 251547282*g*h*k^4 - 78928830*k^6)*s^6 - 1/5*(102942875*g^4*h^4 + 289560600*g^3*h^3*k^2 - 1220229360*g^2*h^2*k^4 - 1366017696*g*h*k^6 - 127545840*k^8)*s^4 - 1/15*(73530625*g^5*h^5 - 197182125*g^4*h^4*k^2 + 669173400*g^3*h^3*k^4 + 1046873160*g^2*h^2*k^6 + 1323642384*g*h*k^8 + 229582512*k^10)*s^2))^(1/3)*(490*g*h + 459*k^2 + 588*s^2) + 7*(-42875*g^3*h^3 + 103950*g^2*h^2*k^2 + 131220*g*h*k^4 + 157464*k^6 + 296352*s^6 + 504*(980*g*h + 27*k^2)*s^4 + 18*(85